In [ ]:
import cv2
import requests
from IPython.display import HTML
import cognitive_face as CF
from io import BytesIO
from PIL import Image, ImageDraw
import xlrd

data = xlrd.open_workbook("/home/bkic611/face_detect/face_detected_filter.xls")
list = data.sheet_by_index(0)


In [2]:

region = "westcentralus"
subscription_key = "d6ab8eba9c094e9592847e406db1a4fd"
assert subscription_key
face_api_url = 'https://westcentralus.api.cognitive.microsoft.com/face/v1.0'


In [3]:
headers = { 'Ocp-Apim-Subscription-Key': 'd6ab8eba9c094e9592847e406db1a4fd' }
headers['Content-Type'] = 'application/octet-stream'

params = {
    'returnFaceId': 'true',
    'returnFaceLandmarks': 'false',
    #'returnFaceAttributes': 'age,gender,headPose,smile,facialHair,glasses,emotion,hair,makeup,occlusion,accessories,blur,exposure,noise',
}
index = 62;

In [4]:
for i in range(1,20):
    channel = int(list.cell(index+1, 0).value)
    time = int(list.cell(index+1, 1).value)
    frame_indx = int(list.cell(index+1, 2).value)

    
    name = '/home/bkic611/face_detect/raw_img/' + channel + '/'+ time + '_fr%d' % frame_indx + '.png'   
    path_to_file = name #path den file local
    
    img = cv2.imread(path_to_file)
    with open(path_to_file, 'rb') as f:
        data = f.read()
    
    response = requests.request('post', "https://{}.api.cognitive.microsoft.com/face/v1.0/detect".format(region), data=data, headers=headers, params=params)

    faces = response.json()
    index = index + 1;

    for face in faces:
        rect = face['faceRectangle']
        x = rect['left']
        y = rect['top']
        w = rect['width']
        h = rect['height']
        cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #roi_gray = gray[y:y + h, x:x + w]
        roi_color = img[y:y + h, x:x + w]
        #cv2.imshow(img)
        name_dt = '/home/bkic611/face_detect/apidetect_img/' + channel + '/'+ time + '_fr%d' % frame_indx + '.png'
        cv2.imwrite(name_dt,img)

In [6]:
#Convert width height to a point in a rectangle
def getRectangle(faceDictionary):
    rect = faceDictionary['faceRectangle']
    left = rect['left']
    top = rect['top']
    bottom = left + rect['height']
    right = top + rect['width']
    return ((left, top), (bottom, right))

#Download the image from the url
img = Image.open(path_to_file)

#For each face returned use the face rectangle and draw a red box.
draw = ImageDraw.Draw(img)
# cv2.imshow('detected',img)

for face in faces:
    draw.rectangle(getRectangle(face), outline='blue')

#Display the image in the users default image browser.
img.show()
with open('picture_out.png', 'wb') as f:
    f.write(data)